# Build a Travel planner agentic system using the Granite-3-8B-Instruct model in watsonx.ai


**Author**: Manoj Jahgirdar

In this tutorial, you will create a Travel planner agent with LangChain agents using the IBM [Granite-3-8B-Instruct model](https://www.ibm.com/granite) now available on [watsonx.ai](https://www.ibm.com/products/watsonx-ai) that can answer complex queries about planning a trip.


# Overview of the use case

## What is the problem?
Planning a trip can be a daunting task, especially when you have to consider various factors such as weather, attractions, and transportation. A travel planner agent can help you plan your trip by providing personalized recommendations based on your preferences and constraints. A normal llm cannot get current events and weather information. We'll see one such example where the llm fails suggests places that are closed recently and is completely clueless about the weather.


## What are AI agents?

At the core of this use case is [artificial intelligence (AI)](https://www.ibm.com/topics/artificial-intelligence) agents. An [AI agent](https://www.ibm.com/think/topics/ai-agents) is a system that uses LLM to understand “the path to follow” in order to answer a user’s query accurately. Since LLMs are becoming good at reasoning, they can be leveraged to "create a plan of action" and execute the plan of action by allowing the agent to  use a set of tools. In this use case, the agent uses Wikipedia and OpenMeteo tools to get the current events and weather information to provide a more personalized answer.

# Prerequisites

You need an [IBM Cloud® account](https://cloud.ibm.com/registration) to create a [watsonx.ai™](https://www.ibm.com/products/watsonx-ai) project.

# Steps

## Step 1. Set up your environment

While you can choose from several tools, this tutorial walks you through how to set up an IBM account to use a Jupyter Notebook. 

1. Log in to [watsonx.ai](https://dataplatform.cloud.ibm.com/registration/stepone?context=wx&apps=all) using your IBM Cloud account.

2. Create a [watsonx.ai project](https://www.ibm.com/docs/en/watsonx/saas?topic=projects-creating-project).

	You can get your project ID from within your project. Click the **Manage** tab. Then, copy the project ID from the **Details** section of the **General** page. You need this ID for this tutorial.

3. Create a [Jupyter Notebook](https://www.ibm.com/docs/en/watsonx/saas?topic=editor-creating-managing-notebooks).

This step will open a Notebook environment where you can copy the code from this tutorial.  Alternatively, you can download this notebook to your local system and upload it to your watsonx.ai project as an asset.

## Step 2. Set up a Watson Machine Learning (WML) service instance and API key.

1. Create a [Watson Machine Learning](https://cloud.ibm.com/catalog/services/watson-machine-learning) service instance (select your appropriate region and choose the Lite plan, which is a free instance).


2. Generate an [API Key in WML](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-authentication.html). 


3. Associate the WML service to the project that you created in [watsonx.ai](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/assoc-services.html). 


## Step 3. Install and import relevant libraries and set up your credentials

We'll need a few libraries and modules for this tutorial. Make sure to import the following ones; if they're not installed, you can resolve this with a quick pip installation. 

Common Python frameworks for building agentic RAG systems include LangChain and LlamaIndex. In this tutorial, we will be using LangChain.  


In [ ]:
# installations
%pip install -q git+https://github.com/ibm-granite-community/utils \
    langchain \
    langchain-ibm \
    langchain_community \
    ibm-watsonx-ai \
    ibm_watson_machine_learning \
    wikipedia

In [2]:
# imports
from langchain_ibm import WatsonxLLM
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts import PromptTemplate
from langchain.tools.render import render_text_description_and_args
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents import AgentExecutor
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

Set up your credentials. Please store your `PROJECT_ID` and `APIKEY` in a separate `.env` file in the same level of your directory as this notebook.

In [3]:
from ibm_granite_community.notebook_utils import get_env_var

credentials = {
    "url": get_env_var("WATSONX_URL"),
    "apikey": get_env_var("WATSONX_APIKEY")
}
project_id = get_env_var("WATSONX_PROJECT_ID")

## Step 4. Initialization a basic agent with no tools

This step is important as it will produce a clear example of an agent's behavior with and without external data sources. Let's start by setting our parameters.

The model parameters available can be found [here](https://ibm.github.io/watson-machine-learning-sdk/model.html). We experimented with various model parameters, including temperature, minimum and maximum new tokens and stop sequences. Learn more about model parameters and what they mean in the [watsonx docs](https://www.ibm.com/docs/en/watsonx/saas). It is important to set our `stop_sequences` here in order to limit agent hallucinations. This tells the agent to stop producing further output upon encountering particular substrings. In our case, we want the agent to end its response upon reaching an observation and to not hallucinate a human response. Hence, one of our stop_sequences is `'Human:'` and another is `Observation` to halt once a final response is produced.

For this tutorial, we suggest using IBM's Granite 3 8B Instruct model as the LLM to achieve similar results. You are free to use any AI model of your choice. The foundation models available through watsonx can be found [here](https://www.ibm.com/products/watsonx-ai/foundation-models). The purpose of these models in LLM applications is to serve as the reasoning engine that decides which actions to take.

In [4]:
llm = WatsonxLLM(
    # @@@ahoaho XXX
    # model_id="ibm/granite-3-8b-instruct",
    model_id="ibm/granite-3-2-8b-instruct-preview-rc",
    url=credentials.get("url"),
    apikey=credentials.get("apikey"),
    project_id=project_id,
    params={
        GenParams.DECODING_METHOD: "greedy",
        GenParams.MIN_NEW_TOKENS: 5,
        GenParams.MAX_NEW_TOKENS: 800,
        GenParams.STOP_SEQUENCES: ["Human:", "Observation"],
    },
)

We'll set up a prompt template in case you want to ask multiple questions. 

In [5]:
template = "Answer the {input} accurately. If you do not know the answer, simply say you do not know."
prompt = PromptTemplate.from_template(template)

And now we can set up a chain with our prompt and our LLM. This allows the generative model to produce a response.

In [6]:
agent = prompt | llm

Let's test to see how our agent responds to a basic query. 

In [7]:
print(agent.invoke({"input": "How is the weather in New York?"}))



I'm sorry for the inconvenience, but I don't have real-time data or location tracking capabilities. Please check a reliable weather forecast source for the current weather in New York.


The agent failed to answer the above query as it cannot get the current events.

In [8]:
print(agent.invoke({"input": "I am planning a trip to New York City next week. Can you gather information about the best places to visit, provide a weather forecast, and recommend activities based on current events and conditions?"}))



Sure, I'd be happy to help you plan your trip to New York City!

**Best Places to Visit:**

1. **Statue of Liberty & Ellis Island**: A must-visit for its historical significance. You can take a ferry from Battery Park to explore both sites.

2. **Central Park**: A sprawling urban park perfect for a leisurely stroll, bike ride, or picnic. Don't miss the Central Park Zoo and Bethesda Terrace.

3. **Metropolitan Museum of Art**: One of the world's largest and finest art museums, housing over two million works of art spanning 5,000 years.

4. **Times Square**: The bustling heart of NYC, filled with bright lights, Broadway theaters, and diverse dining options.

5. **9/11 Memorial & Museum**: A poignant tribute to the victims of the 9/11 attacks, offering a profound and educational experience.

6. **Brooklyn Bridge**: Walk or bike across this iconic suspension bridge for stunning views of the city skyline.

7. **Empire State Building**: Visit the iconic Art Deco skyscraper for panoramic vi

Evidently, The LLM gave a generic response and to its knowledge it was last updated upon and also hallucinated a response. In the next session lets solve the knowledge cut off problem and build a system around the LLM so that it gives us more personalized answer.

## Step 5. Setup Tools

The main tools required for the agentic system to get the current events. The tools are:
- **Wikipedia Tool:** To get the attractions
- **OpenMeteo Tool:** To get the weather details 

In [9]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.tools import Tool

class WikipediaTool:
    def get_information(self, query):
        wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
        response = wikipedia.run(query)
        return response

    def get_tool(self):
        return Tool.from_function(
            func=self.get_information,
            name="WikipediaTool",
            description="Use this tool to get attractions of a city."
        )

Next, load the documents using LangChain `WebBaseLoader` for the URLs we listed. We'll also print a sample document to see how it loaded.

In [10]:
import requests
from urllib.parse import quote  # For URL encoding city names
from langchain.tools import Tool

class OpenMeteoTool:
    def get_coordinates(self, city_name):
        # URL encode the city name to handle spaces and special characters
        encoded_city_name = quote(city_name)

        geocode_url = f"https://nominatim.openstreetmap.org/search?q={encoded_city_name}&format=json"

        headers = {
            'User-Agent': 'MyWeatherApp/1.0 (Geocoding and Weather Service)'  # Generic app description
        }

        response = requests.get(geocode_url, headers=headers)

        if response.status_code == 200:
            data = response.json()

            if data:
                latitude = data[0].get('lat')
                longitude = data[0].get('lon')

                # Ensure latitude and longitude were found
                if latitude and longitude:
                    return latitude, longitude
                else:
                    raise ValueError(f"Coordinates not found for '{city_name}'.")
            else:
                raise ValueError(f"No data returned for city '{city_name}'.")
        else:
            raise Exception(f"Nominatim API returned an error: {response.status_code}")

    def get_weather(self, city_name):
        try:
            # Fetch coordinates using the get_coordinates method
            lat, lon = self.get_coordinates(city_name)

            # Fetch weather information from Open-Meteo API using the coordinates
            weather_url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true"
            weather_response = requests.get(weather_url)

            if weather_response.status_code == 200:
                weather_data = weather_response.json()
                current_weather = weather_data.get('current_weather')

                if current_weather:
                    response = f"Current temperature in {city_name} is {current_weather['temperature']}°C, with wind speed of {current_weather['windspeed']} m/s and it is { 'day' if current_weather['is_day'] == 1 else 'night'} time."
                    return response
                else:
                    raise Exception("Weather data not available.")
            else:
                raise Exception(f"Open-Meteo API returned an error: {weather_response.status_code}")
        except Exception as e:
            return str(e)

    def get_tool(self):
        return Tool.from_function(
            func=self.get_weather,
            name="OpenMeteoTool",
            description="Use this tool to get weather information of a city."
        )


In [11]:
tools = [WikipediaTool().get_tool(), OpenMeteoTool().get_tool()]

## Step 6. Establish the prompt template

Next, we will set up a new prompt template to ask multiple questions. This template is more complex. It is referred to as a [structured chat prompt](https://api.python.langchain.com/en/latest/agents/langchain.agents.structured_chat.base.create_structured_chat_agent.html#langchain-agents-structured-chat-base-create-structured-chat-agent) and can be used for creating agents that have multiple tools available. In our case, the tool we are using was defined in Step 5. The structured chat prompt will be made up of a `system_prompt`, a `human_prompt` and our tools. 

First, we will set up the `system_prompt`. This prompt instructs the agent to print its "thought process," which involves the agent's subtasks, the tools that were used and the final output. This gives us insight into the agent's function calling. The prompt also instructs the agent to return its responses in JSON Blob format.

In [12]:
system_prompt = """<|start_of_role|>system<|end_of_role|>Respond to the human as helpfully and accurately as possible.
You have access to the following tools:<|end_of_text|>
<|start_of_role|>tools<|end_of_role|>
{tools}
<|end_of_text|>
<|start_of_role|>system<|end_of_role|>
Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).
Valid "action" values: "Final Answer" or {tool_names}
Provide only ONE action per $JSON_BLOB, as shown:"
```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```
Follow this format:
Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}
Begin! Reminder to ALWAYS respond with a valid json blob of a single action.
ALways Remember to respond in a structured manner with proper bullet points and lists.
Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation
<|end_of_text|>"""

In the following code, we are establishing the `human_prompt`. This prompt tells the agent to display the user input followed by the intermediate steps taken by the agent as part of the `agent_scratchpad`.

In [13]:
human_prompt = """<|start_of_role|>user<|end_of_role|>{input}<|end_of_text|>
{agent_scratchpad}
(reminder to always respond in a JSON blob)"""

In [14]:
assistant_prompt = """<|start_of_role|>assistant<|end_of_role|>"""

Next, we establish the order of our newly defined prompts in the prompt template. We create this new template to feature the `system_prompt` followed by an optional list of messages collected in the agent's memory, if any, and finally, the `human_prompt` which includes both the human input and `agent_scratchpad`.

In [15]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", human_prompt),
        ("assistant", assistant_prompt),
    ]
)

Now, let's finalize our prompt template by adding the tool names, descriptions and arguments using a [partial prompt template](https://python.langchain.com/v0.1/docs/modules/model_io/prompts/partial/). This allows the agent to access the information pertaining to each tool including the intended use cases and also means we can add and remove tools without altering our entire prompt template.

In [16]:
prompt = prompt.partial(
    tools=render_text_description_and_args(list(tools)),
    tool_names=", ".join([t.name for t in tools]),
)

## Step 7. Set up the agent's memory and chain

An important feature of AI agents is their memory. Agents are able to store past conversations and past findings in their memory to improve the accuracy and relevance of their responses going forward. In our case, we will use LangChain's `ChatMessageHistory` that simply store chat history in the prompt and send it with every user question to the LLM.

In [17]:
message_history = ChatMessageHistory()

And now we can set up a chain with our agent's scratchpad, memory, prompt and the LLM. The AgentExecutor class is used to execute the agent. It takes the agent, its tools, error handling approach, verbose parameter and memory as parameters.

In [18]:
chain = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"]),
    )
    | prompt
    | llm
    | JSONAgentOutputParser()
)

agent_executor_chat = AgentExecutor(
    agent=chain, tools=tools, handle_parsing_errors=True, verbose=True
)

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor_chat,
    get_session_history=lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

## Step 8. Generate responses with the travel planner agent

We are now able to ask the agent questions. Recall the agent's previous inability to provide us with information pertaining to the weather and attractions. Now that the agent has its tools available to use, let's try asking the same questions again. 

In [19]:
answer1 = agent_with_chat_history.invoke(
        {"input": "How is the weather in New York?"},
        config={"configurable": {"session_id": "watsonx"}}
)



> Entering new AgentExecutor chain...
Thought: The user wants to know the weather in New York. I will use the OpenMeteoTool to get this information.

Action:
```
{
  "action": "OpenMeteoTool",
  "action_input": "New York"
}
```

ObservationCurrent temperature in New York is 1.8°C, with wind speed of 6.8 m/s and it is day time.```
{
  "action": "Final Answer",
  "action_input": "The current temperature in New York is 1.8°C, with wind speed of 6.8 m/s and it is day time."
}
```

> Finished chain.


In [20]:
print(answer1['output'])

The current temperature in New York is 1.8°C, with wind speed of 6.8 m/s and it is day time.


Great! The agent used its available OpenMeteo tool to return the weather conditions of New York!

In [21]:
answer2 = agent_with_chat_history.invoke(
        {"input": "I am planning a trip to New York next week. Can you gather information about the best places to visit, provide a weather forecast, and recommend activities based on current events and conditions?"},
        config={"configurable": {"session_id": "watsonx"}}
)



> Entering new AgentExecutor chain...
Thought: The user wants to know about the best places to visit in New York, the weather forecast, and activities based on current events and conditions. I will use the WikipediaTool to find the best places to visit and the OpenMeteoTool to get the weather forecast.

Action:
```
{
  "action": "WikipediaTool",
  "action_input": "Best places to visit in New York"
}
```

ObservationPage: The Band's Visit (musical)
Summary: The Band's Visit is a stage musical with music and lyrics by David Yazbek and a book by Itamar Moses, based on the 2007 Israeli film of the same name. The musical opened on Broadway at the Ethel Barrymore Theatre in November 2017, after its off-Broadway premiere at the Atlantic Theater Company in December 2016.
The Band's Visit has received critical acclaim. Its off-Broadway production won several major awards, including the 2017 Obie Award for Musical Theatre, as well the year's New York Drama Critics' Circle Award for Best Musica

In [22]:
print(answer2['output'])

Based on the information gathered, here are the details for your trip to New York:

**Best Places to Visit:**
New York City is a vibrant metropolis with a rich history and diverse culture. Some of the top attractions include:
1. Statue of Liberty and Ellis Island: Iconic symbols of freedom and immigration.
2. Central Park: A sprawling urban park offering various recreational activities.
3. Metropolitan Museum of Art: One of the world's largest and finest art museums.
4. Times Square: A bustling hub of entertainment, shopping, and dining.
5. Broadway: Home to numerous world-class theaters and musicals, such as The Band's Visit.

**Weather Forecast:**
The current temperature in New York is 1.8°C, with wind speed of 6.8 m/s and it is day time.

**Activities Based on Current Events and Conditions:**
Given the current weather, consider visiting indoor attractions like museums, theaters, and shopping centers. You can also enjoy a Broadway show, visit the Metropolitan Museum of Art, or explor

As you can see, the agent used its Wikipedia tool to get the top 10 attractions and OpenMeteo tool to get the weather information. As a result hallucination is reduced and the agent is able to provide a more personalized answer.

## Summary

In this tutorial, you created a travel planner agent using LangChain in python with watsonx. The LLM you worked with was the IBM Granite-3-8B-Instruct model. The sample output is important as it shows the significance of this [generative AI](https://www.ibm.com/topics/generative-ai) advancement. The AI agent was successfully able to retrieve relevant information via the `WikipediaTool` and `OpenMeteoTool` tools, update its memory with each interaction and output appropriate responses. It is also important to note the agent's ability to determine whether tool calling is appropriate for each specific task. When the agent had the information necessary to answer the input query, it did not use any tools for question answering. 

For more AI agent content, we encourage you to check out our [Use watsonx.ai and LangChain Agents to perform E-commerce Analytics](https://github.com/manojjahgirdar/ibm-granite-recipes/blob/main/LLM_Agent_for_E-commerce_Analytics.ipynb) This Agentic solution for e-commerce analytics provides businesses with actionable insights into customer behavior and product interactions. The agent enables a deeper understanding of purchasing patterns and customer sentiment. It empowers businesses to enhance personalization, optimize product offerings, and improve customer satisfaction by delivering tailored insights that drive informed decision-making. This solution transforms raw data into valuable business intelligence, helping companies stay competitive and responsive to customer needs.